# Rapport du projet de programmation

## Introduction

Le projet de programmation que nous avons entrepris se veut proposer des formes d'analyses et de réponses au sujet des thématiques et problématiques qui touchent l'utilisation des réseaux sociaux à des fins d'information, notamment Twitter. En effet, la construction de véritables empires médiatiques est croissante et tend à se normaliser. Le choix de s'intéresser
Quelle attitude des utilisateurs face à cela ? 

## Scrapping 
Afin de construire les bases de données nécessaires à notre analyse, nous avons décidé de mettre en place une méthode de scrapping en utilisant le package selenium. Cela nous est paru nécessaire d'utiliser cette méthode dans notre projet pour obtenir les types d'informations spécifiques que nous voulions recueillir. 
Ainsi, dans la base de données, nous avons en colonne :
- le nom de l'utilisateur
- 

## NLP

### Code

### Entraînement
- Choix 

## Visualisation


On commence par installer les différentes bibliothèques qui seront nécessaires à l'exécution du code :

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install datetime

Une fois ceci fait, la première étape va consister à scrapper twitter. La façon dont nous avons décidé de scrapper Twitter a été la suivante :
- Tout d'abord, nous avons souhaité récolté les tweets dans leur ordre de publication, c'est-à-dire dans l'onglet "Récents" de la page de recherche de Twitter. Pour cela, nous avons créé un compte Twitter : l'utilisateur et le mot de passe sont remplis lors de l'exécution du code. 
- Ensuite, nous ne voulions que les tweets en anglais, ce qui était nécessaire pour l'étape de NLP, donc nous avons appliqué le filtre **nom du filtre**.
- bref philipa tu pourras expliquer 

**ATTENTION : par pitié ne pas exécuter ce code ça prend littéralement des heures**

In [ ]:
CODE SCRAPPING

Une fois le code exécuté, nous nous retrouvons avec une base de données en format excel **Nettoyer la table ? quelles dates on garde ? mettre des 0 là où y a des trous => mieux si philipa s'en charge?**. Nous avons ainsi en colonnes blablabla. **présenter une sortie du tableau excel ?**

In [10]:
import pandas as pd

# Modifier les paramètres pour afficher plus de contenu dans chaque cellule
pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des colonnes
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
pd.set_option('display.width', 1000)  # Ajuste la largeur totale de l'affichage

# Charger et afficher le fichier Excel
df = pd.read_excel('tweets_macron_ecologie.xlsx')
df.head(15)


,Username,Date,Content
0,HBHANTI Macron Verts Gauches Woke !,6m,Et dépenser moins en évitant sans cesse les taxes et impôts c'est possible aussi à Bordeaux sans cette écologie punitive ??
1,Titube,1h,"Vous pourriez respecter le vote des électeurs américains. Les français ont voté Macron . On ne peut donc pas se permettre de donner des leçons de démocratie, d'ecologie, de justice ou d'économie."
2,Lisa Sister,2h,Vous êtes en parfaite connexion avec Macron qui applaudit l'élection de Trump. C'est très affligeant mais cela confirme la personne politique que vous êtes. Mais bien entendu l'écologie que vous représentez n'est pas celle de la majorité des écologistes.\nVive Marine Tondelier
3,Foudroyeur2Naine,6h,Je vais te faire un exemple pour ton petit cervelet (comme à l’école) : \nMacron est climatosceptique (Ici Trump)\nAntoine Armand (Ici Elon musk) ne l’est pas. Oui ? Donc ? \nMinistre de l’économie quel est le rapport avec l’écologie ?
4,Nathalie BIHET,7h,"Sache que je ne vote que lorsque le parti animaliste est représenté, je n'ai jamais voté Macron. Trump déteste l'écologie et les animaux, et étant végan j'ai beaucoup de mal avec lui tout comme avez Macron. Enfin bref chacun ses idées"
5,Florence Regnier,8h,"Non, pas de la montée. De sa présence au 2nd tour. et de ce fait de la réélection de Macron, pour lequel vous avez revoté. LFI n'a pas attendu les écolos pour faire de l'écologie."
6,Zodiaquue,10h,"Qu'est ce que tu es con et haineux qvec ton mépris imminde. Tu as fait réélire Macron et tu nous parles de débilité, tu as jamais lu un programme de ta vie, tu fais tellement pitié.\nSinon,l'écologie c'est une priorité pour Musk qui était bridé par le gouvernement Biden."
7,jeffe50,11h,"Vivement 2027 qu’on dégage LFI et toutes la bande à Macron, avec leur putain d’écologie ils on détruit l’industrie européenne, mis la France en faillite, mais ils ont de la merde dans les yeux, mais ils croient toujours que l’avenir c’est eux"
8,FreeGaet,12h,"Oui.\nFaut arrêter de croire que Macron est de droite.\n\nTrump veut retirer tous les droits aux migrants sauf celui de rentrer chez eux, il veut supprimer les subventions aux universités US gangrenées par le wokisme et n'a pas la moindre considération pour l'écologie."
9,Xedo,13h,"Même si son passage a l’écologie n’est pas glorieux, elle ne porte pas en tant que ministre la déroute du nucléaire français, imputable AUSSI aux présidents de droite, Chirac Sarko, Hollande, politique de droite et Macron qui est incapable de corriger le tir..,"


Voilà à quoi ressemble notre sortie : l'étape d'après est le NLP.

## NLP

## Visualisation
Une fois la base de données nettoyée après avoir été obtenue par scrapping, l'objectif est de l'organiser et de créer les variables nécessaires à la création d'indices, eux-mêmes nécessaires à la réalisation de statistiques descriptives et d'analyses.

### Rajout d'une colonne YearWeek
Les dates que nous obtenons directement après avoir scrappé ne sont pas dans un format "date" qui simplifie les calculs. Nous utilisons donc le module *datetime* de Python, notamment la fonction *datetime.strptime* pour convertir une chaîne de caractères représentant une date ou une heure en un objet datetime. En effet, nous avons été particulièrement vigilants au fait que notre code devait faire en sorte d'assigner la bonne date aux dates apparaissant sous le format "6mn", "1h" : dans notre code, nous faisons en sorte de prendre en compte la date suivant la précédente. Ainsi, nous avons pu régler des problèmes de date d'exécution du code.
Nous avons choisi d'analyser nos données en les groupant par semaine, ce qui permet d'avoir une analyse assez fine des tendances sans être trop exposés aux fluctuations journalières.

Dans le code suivant, nous avons créé une nouvelle table avec un rajout de la colonne yearweek. 


code 0_rap_convertdate

## Premiers graphiques
Une fois ceci fait, nous pouvons sortir nos premiers graphiques :
- d'abord le **nombre de tweets par semaine**, ce qui permet de relever certaines tendances
- puis sur un autre graphique nous représentons le **nombre de likes, de retweets et de réponses par semaine** en **valeur absolue** puis en **valeur pondérée** par le nombre de tweets.

In [ ]:
code 1_rap_nbrtweets
code 2_rap_nbrlikes

CONCLUSION de ces premiers éléments


- ensuite, nous avons décidé de **créer différents indices** pour aborder autrement les données. 
1. un taux d'engagement égal à la somme de likes, de retweets, de réponses le tout sur le nombre de vues, exprimé en pourcentages. Nous mettons en place un taux d'engagement par semaine. 
2. un taux de croissance des interactions : nous nous sommes ici restreints à la variation de likes d'une semaine à l'autre
**PONDERER PAR LE NOMBRE DE TWEETS**

In [ ]:
code 3_rap_indices

Une fois ces indices créés, nous les représentons sur un même graphique :

In [ ]:
4_graph_indices